In [50]:
import numpy as np
import math
import matplotlib.pyplot as plt
import operator

from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

In [51]:
from sklearn.datasets import load_breast_cancer

breast = load_breast_cancer()

X = breast['data']
y = breast['target']

np.random.seed(100)
p = np.random.permutation(len(X))
X, y = X[p], y[p]

X_train, y_train = X[:400], y[:400]
X_val, y_val = X[400:500], y[400:500]
X_test, y_test = X[500:], y[500:]


def distanceFunc(metric_type, vec1, vec2):
    """
    Computes the distance between two d-dimension vectors. 
    
    Please DO NOT use Numpy's norm function when implementing this function. 
    
    Args:
        metric_type (str): Metric: L1, L2, or L-inf
        vec1 ((d,) np.ndarray): d-dim vector
        vec2 ((d,)) np.ndarray): d-dim vector
    
    Returns:
        distance (float): distance between the two vectors
    """

    diff = vec1 - vec2
    if metric_type == "L1":
        return np.sum([np.abs(i) for i in diff])

    if metric_type == "L2":
        return np.sqrt(np.sum([i**2 for i in diff]))
        
    if metric_type == "L-inf":
        return np.max([np.abs(i) for i in diff])


def computeDistancesNeighbors(K, metric_type, X_train, y_train, sample):
    """
    Compute the distances between every datapoint in the train_data and the 
    given sample. Then, find the k-nearest neighbors.
    
    Return a numpy array of the label of the k-nearest neighbors.
    
    Args:
        K (int): K-value
        metric_type (str): metric type
        X_train ((n,p) np.ndarray): Training data with n samples and p features
        y_train : Training labels
        sample ((p,) np.ndarray): Single sample whose distance is to computed with every entry in the dataset
        
    Returns:
        neighbors (list): K-nearest neighbors' labels
    """

    dis = [distanceFunc(metric_type, sample, X_train[i, :]) for i in range(X_train.shape[0])]
    idx = sorted(range(X_train.shape[0]), key=dis.__getitem__)
    neighbors = [y_train[idx[i]] for i in range(K)]
    
    return neighbors


def Majority(neighbors):
    """
    Performs majority voting and returns the predicted value for the test sample.
    
    Since we're performing binary classification the possible values are [0,1].
    
    Args:
        neighbors (list): K-nearest neighbors' labels
        
    Returns:
        predicted_value (int): predicted label for the given sample
    """
    
    counts = [neighbors.count(i) for i in neighbors]
    predicted_value = neighbors[np.argmax(counts)]

    return predicted_value


def KNN(K, metric_type, X_train, y_train, X_val):
    """
    Returns the predicted values for the entire validation or test set.
    
    Please DO NOT use Scikit's KNN model when implementing this function. 

    Args:
        K (int): K-value
        metric_type (str): metric type
        X_train ((n,p) np.ndarray): Training data with n samples and p features
        y_train : Training labels
        X_val ((n, p) np.ndarray): Validation or test data
        
    Returns:
        predicted_values (list): output for every entry in validation/test dataset 
    """
    
    predictions = []
    for i in range(X_val.shape[0]):
        sample = X_val[i, :]
        neighbors = computeDistancesNeighbors(K, metric_type, X_train, y_train, sample)

        predictions.append(Majority(neighbors))

    return predictions


def evaluation(predicted_values, actual_values):
    """
    Computes the accuracy of the given datapoints.
    
    Args:
        predicted_values ((n,) np.ndarray): Predicted values for n samples
        actual_values ((n,) np.ndarray): Actual values for n samples
    
    Returns:
        accuracy (float): accuracy
    """
    
    return accuracy_score(predicted_values, actual_values)


def main():
    """
    Calls the above functions in order to implement the KNN algorithm.
    
    Test over the following range K = 3,5,7 and all three metrics. 
    In total you will have nine combinations to try.
    
    PRINTS out the accuracies for the nine combinations on the validation set,
    and the accuracy on the test set for the selected K value and appropriate norm.
    
    REMEMBER: You have to report these values by populating the Table 1.
    """
    
    ## Complete this function
    
    K = [3, 5, 7]
    norm = ["L1", "L2", "L-inf"]
    
    print("<<<<VALIDATION DATA PREDICTIONS>>>>")

    values = []
    for i in K:
        print(f'for K = {i}:')
        print(f'\tL1\tL2\tL-inf')

        values = []
        predictions = KNN(i, "L1", X_train, y_train, X_val)
        acc = evaluation(predictions, y_val)
        values.append(acc)
        
        predictions = KNN(i, "L2", X_train, y_train, X_val)
        acc = evaluation(predictions, y_val)
        values.append(acc)

        predictions = KNN(i, "L-inf", X_train, y_train, X_val)
        acc = evaluation(predictions, y_val)
        values.append(acc)
        print(f'acc\t{values[0]}\t{values[1]}\t{values[2]}')

    print("\n<<<<TEST DATA PREDICTIONS>>>>")

    values.clear()
    for i in K:
        print(f'for K = {i}:')
        print(f'\tL1\tL2\tL-inf')

        predictions = KNN(i, "L1", X_train, y_train, X_test)
        acc = evaluation(predictions, y_test)
        values.append(acc)
        
        predictions = KNN(i, "L2", X_train, y_train, X_test)
        acc = evaluation(predictions, y_test)
        values.append(acc)

        predictions = KNN(i, "L-inf", X_train, y_train, X_test)
        acc = evaluation(predictions, y_test)
        values.append(acc)
        print(f'acc\t{values[0]}\t{values[1]}\t{values[2]}')

In [52]:
if __name__ == "__main__":
    main()

<<<<VALIDATION DATA PREDICTIONS>>>>
for K = 3:
	L1	L2	L-inf
acc	0.94	0.95	0.94
for K = 5:
	L1	L2	L-inf
acc	0.94	0.93	0.94
for K = 7:
	L1	L2	L-inf
acc	0.93	0.92	0.93

<<<<TEST DATA PREDICTIONS>>>>
for K = 3:
	L1	L2	L-inf
acc	0.8840579710144928	0.8840579710144928	0.8985507246376812
for K = 5:
	L1	L2	L-inf
acc	0.8840579710144928	0.8840579710144928	0.8985507246376812
for K = 7:
	L1	L2	L-inf
acc	0.8840579710144928	0.8840579710144928	0.8985507246376812


Answer the following questions here:

1. How could having a larger dataset influence the performance of KNN?

2. Tabulate your results from `main()` in the table provided.

3. Finally, mention the best K and the norm combination you have settled upon and report the accuracy on the test set using that combination.

1. KNN would have a better performance since a greate deviation may exist when the dataset is small. Coincidence can influence the results a lot in small datasets.

2. The result is shown above.

3. The best combination of my result is {K=3, L2}. The accuracy is 0.95.